In [1]:
# https://dplyr.tidyverse.org/reference/tibble.html
# https://tibble.tidyverse.org/reference/tribble.html

from datar import f
from datar.tibble import tibble, tibble_row, tribble, fibble
from datar.base import diag
from datar.stats import runif

%run nb_helpers.py
nb_header(tibble, tibble_row, tribble, fibble, book='tibble')

### # tibble  

##### Constructs a data frame

##### Args:
&emsp;&emsp;`*args`: and  
&emsp;&emsp;`**kwargs`: A set of name-value pairs.  
&emsp;&emsp;`_name_repair`: treatment of problematic column names:  
&emsp;&emsp;&emsp;&emsp;- "minimal": No name repair or checks, beyond basic existence,

&emsp;&emsp;&emsp;&emsp;- "unique": Make sure names are unique and not empty,

&emsp;&emsp;&emsp;&emsp;- "check_unique": (default value), no name repair,
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;but check they are unique,  

&emsp;&emsp;&emsp;&emsp;- "universal": Make the names unique and syntactic

&emsp;&emsp;&emsp;&emsp;- a function: apply custom name repair

&emsp;&emsp;`_rows`: Number of rows of a 0-col dataframe when args and kwargs are  
&emsp;&emsp;&emsp;&emsp;not provided. When args or kwargs are provided, this is ignored.  

##### Returns:
&emsp;&emsp;A constructed dataframe  


### # tibble_row  

##### Constructs a data frame that is guaranteed to occupy one row.

Scalar values will be wrapped with `[]`  

##### Args:
&emsp;&emsp;`*args`: and  
&emsp;&emsp;`**kwargs`: A set of name-value pairs.  
&emsp;&emsp;`_name_repair`: treatment of problematic column names:  
&emsp;&emsp;&emsp;&emsp;- "minimal": No name repair or checks, beyond basic existence,

&emsp;&emsp;&emsp;&emsp;- "unique": Make sure names are unique and not empty,

&emsp;&emsp;&emsp;&emsp;- "check_unique": (default value), no name repair,
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;but check they are unique,  

&emsp;&emsp;&emsp;&emsp;- "universal": Make the names unique and syntactic

&emsp;&emsp;&emsp;&emsp;- a function: apply custom name repair

##### Returns:
&emsp;&emsp;A constructed dataframe  


### # tribble  

##### Create dataframe using an easier to read row-by-row layout

Unlike original API that uses formula (`f.col`) to indicate the column  
names, we use `f.col` to indicate them.  

##### Args:
&emsp;&emsp;`*dummies`: Arguments specifying the structure of a dataframe  
&emsp;&emsp;&emsp;&emsp;Variable names should be specified with `f.name`  

##### Examples:
&emsp;&emsp;>>> tribble(  
&emsp;&emsp;>>>     f.colA, f.colB,  
&emsp;&emsp;>>>     "a",    1,  
&emsp;&emsp;>>>     "b",    2,  
&emsp;&emsp;>>>     "c",    3,  
&emsp;&emsp;>>> )  

##### Returns:
&emsp;&emsp;A dataframe  


### # fibble  

##### A function of tibble that can be used as an argument of verbs

Since `tibble` can recycle previous items, for example:  
&emsp;&emsp;>>> df >> tibble(x=1, y=f.x+1)  
&emsp;&emsp;>>> # x y  
&emsp;&emsp;>>> # 1 2  

It gets confused when it is used as an argument of a verb, the we can't tell  
whether `f` if a proxy for the data of the verb or the data frame that  
`tibble` is constructing. So then here is the function to be used as a verb  
argument so `f` refers to the data of the verb. Note that in such a case,  
the items coming in previously cannot be recycled.  

See `tibble` for details.  


In [2]:
a = range(5)
tibble(a, f.a*2)

,a,a
0,0,0
1,1,2
2,2,4
3,3,6
4,4,8


In [3]:
tibble(a, b=f.a * 2, c=1)

,a,b,c
0,0,0,1
1,1,2,1
2,2,4,1
3,3,6,1
4,4,8,1


In [4]:
runif(10).shape

(10,)

In [5]:
tibble(x=runif(10), y=f.x*2)

,x,y
0,0.313694,0.627387
1,0.449833,0.899666
2,0.320806,0.641613
3,0.950275,1.900550
4,0.503762,1.007525
5,0.033943,0.067886
6,0.588497,1.176994
7,0.836514,1.673028
8,0.414526,0.829052
9,0.932544,1.865089


In [6]:
from datar.core.exceptions import NameNonUniqueError
x = 1
try:
    tibble(x, x)
except NameNonUniqueError as err:
    print(err)

Names must be unique: x


In [7]:
tibble(x, x, _name_repair="unique")

[2021-05-06 15:39:39][datar][WARNING] New names:
[2021-05-06 15:39:39][datar][WARNING] * 'x' -> 'x__0'
[2021-05-06 15:39:39][datar][WARNING] * 'x' -> 'x__1'


,x__0,x__1
0,1,1


In [8]:
tibble(x, x, _name_repair="minimal") # duplicated columns collapse

,x,x
0,1,1


In [9]:
a = 1
tibble(a * 1, a * 2, _name_repair="universal")

[2021-05-06 15:39:40][datar][WARNING] New names:
[2021-05-06 15:39:40][datar][WARNING] * 'a * 1' -> 'a___1'
[2021-05-06 15:39:40][datar][WARNING] * 'a * 2' -> 'a___2'


,a___1,a___2
0,1,2


In [10]:
from typing import Iterable
def make_unique(names: Iterable[str]):
    new_names = []
    for name in names:
        name_count = new_names.count(name)
        if name_count == 0:
            new_names.append(name)
        else:
            new_names.append(f'{name}_{name_count}')
    return new_names

tibble(a, a, _name_repair=make_unique)

,a,a_1
0,1,1


In [11]:
def fix_names(name):
    import re
    return re.sub(r'\s+', '_', name)


tibble(a + 1, a + 2, _name_repair = fix_names)

,a_+_1,a_+_2
0,2,3


In [12]:
tibble(x, x, _name_repair=["a", "b"])

,a,b
0,1,1


In [13]:
tibble(
  tibble(
    b = [4,5,6],
    c = [7,8,9]
  ),
  a = range(3),
  d = f.b
)

,b,c,a,d
0,4,7,0,4
1,5,8,1,5
2,6,9,2,6


In [14]:
s = tibble(diag(1, 4))
t = tibble(s.iloc[:, :2], _name_repair=['x', 'y'])
tibble(
  a=range(4),
  b=s,
  c=t
)

,a,b$0,b$1,b$2,b$3,c$x$0,c$x$1
0,0,1,0,0,0,1,0
1,1,0,1,0,0,0,1
2,2,0,0,1,0,0,0
3,3,0,0,0,1,0,0


In [15]:
try:
    tibble(a=range(3), b=range(4))
except ValueError as err:
    print(err)

Length of values (4) does not match length of index (3)


In [16]:
tibble(_dotted = 3, _name_repair=lambda x: x.replace('_', '.'))

,.dotted
0,3


In [17]:
x = 3
tibble(x=1, y=f.x)

,x,y
0,1,1


In [18]:
tibble(x=1, y=x)

,x,y
0,1,3


In [19]:
tribble(
  f.colA, f.colB,
  "a",    1,
  "b",    2,
  "c",    3
)

,colA,colB
0,a,1
1,b,2
2,c,3


In [20]:
tribble(
  f.x,  f.y,
  "a",  [1,2,3],
  "b",  [4,5,6]
)

,x,y
0,a,"[1, 2, 3]"
1,b,"[4, 5, 6]"


In [21]:
tibble_row(a=1, b=[[2,3]])

,a,b
0,1,"[2, 3]"
